# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   <br>
Url: https://github.com/peraltarh/VIUMasterInAI/blob/main/1.Algoritmos%20de%20optimizacion/Seminario_Algoritmos.ipynb<br>
Problema:
> 1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

Número de actores: 10

Número de tomas : 30


Esto se afrontara como un Crew Scheduling Problem o Minimum Cost Crew Assignment, por lo que busque en google la mejor forma de hacerlo es con Greedy Heuristic, algunos incluyen opcionalmente Local search como mejora.

....

(*) La respuesta es obligatoria





                                        

In [1]:
import numpy as np
import pandas as pd
import random
from os import path as osp
# Load dataset
fpath = osp.join("res", "Doblaje.csv")
df = pd.read_csv(fpath, sep=',', header=1, index_col=0, engine='python')

# Drop the last 2 columns and the last row
df = df.iloc[:-2, :-2]

# Convert to a 2D list of integers
#sessions = df.astype(int).values.tolist() # doblaje[][]
sessions = np.array(df)  # Convert to numpy array for easier manipulation

In [44]:
def scenes_organization_gh(sessions, skip_scenes):
    '''Initialize:
    Create a list of all unassigned scenes (0 to 29).
    Create an empty list for each day to store assigned scenes.
    '''

    #sessions = np.array(df)  # Convert to numpy array for easier manipulation
    num_scenes = sessions.shape[0]
    unassigned = set(range(num_scenes))
    days = []

    '''While there are unassigned scenes:

    Pick an unassigned scene as the "seed" for a new day.
    For this day, try to add up to 5 more unassigned scenes that have the most overlap in actors with the current group (i.e., adding them increases the set of unique actors as little as possible).
    Assign these scenes to the current day and remove them from the unassigned list.'''

    while unassigned:
        # Start a new day with the first unassigned scene, this was not a good idea, it ended always with the same result
        #day_scenes = [unassigned.pop()]
        
        # Pick a random scene from unassigned instead
        seed_scene = random.choice(list(unassigned))
        day_scenes = [seed_scene]
        unassigned.remove(seed_scene)
            
        # Get the set of actors present in the first scene
        actors_in_day = set(np.where(sessions[day_scenes[0]] == 1)[0])
        
        # Try to add up to 5 more scenes to this day
        for _ in range(5):
            best_scene = None
            best_increase = None
            for scene in unassigned: # Iterate over unassigned scenes
                # Calculate the increase in unique actors if this scene is added
                # actors_in_day is the set of actors already assigned to this day
                # actors_in_scene is the set of actors in the currently analized scene
                actors_in_scene = set(np.where(sessions[scene] == 1)[0])
                # This was intended to skip scenes have only actors not currently present but it ended up generating worse results
                if skip_scenes and actors_in_scene.difference(actors_in_day) == set():
                    continue  # Skip if no new actors repeat
                increase = len(actors_in_day | actors_in_scene) - len(actors_in_day)
                if best_scene is None or increase < best_increase:
                    best_scene = scene
                    best_increase = increase
            if best_scene is not None:
                day_scenes.append(best_scene)
                actors_in_day |= set(np.where(sessions[best_scene] == 1)[0]) # Update the set of actors |= is union
                unassigned.remove(best_scene)
            else:
                break
        days.append(day_scenes)
    return days

def calc_cost(days,sessions):
    ''' Count:

    For each day, count the unique actors present (i.e., any actor with a 1 in any assigned scene for that day).
    Sum over all days to get the total actor-days.'''

    # Calculate total actor-days
    total_actor_days = 0
    day = 0
    for day_scenes in days:
        actors = set()
        for scene in day_scenes:
            actors |= set(np.where(sessions[scene] == 1)[0])
        total_actor_days += len(actors)
        day += 1 
        #print(f'Scenes of day {day}:{day_scenes}')
    return total_actor_days


total_actor_days = 99999
days = []

print('Forcing always 6 scenes per day:')
for t in range(100): # A bit of an improvement, find the best option in X tries, in this case 100 because it was the number that gave me 27 most of the time
    days_n = scenes_organization_gh(sessions, False)
    total_actor_days_n = calc_cost(days_n,sessions)
    if total_actor_days_n < total_actor_days:
        total_actor_days = total_actor_days_n
        days = days_n
day = 1
for day_scenes in days:
    print(f'Scenes of day {day}:{day_scenes}')
    day += 1

print("Total times I have to pay if these scenes are scheduled (greedy):", total_actor_days)

total_actor_days = 99999
days = []

print('Without Forcing 6 scenes per day:')
for t in range(1):
    days_n = scenes_organization_gh(sessions, True)
    total_actor_days_n = calc_cost(days_n,sessions)
    if total_actor_days_n < total_actor_days:
        total_actor_days = total_actor_days_n
        days = days_n
day = 1
for day_scenes in days:
    print(f'Scenes of day {day}:{day_scenes}')
    day += 1

print("Total times I have to pay if these scenes are scheduled (greedy):", total_actor_days)




Forcing always 6 scenes per day:
Scenes of day 1:[14, 2, 3, 4, 5, 6]
Scenes of day 2:[24, 8, 15, 27, 29, 7]
Scenes of day 3:[16, 18, 22, 13, 17, 23]
Scenes of day 4:[10, 0, 1, 12, 19, 21]
Scenes of day 5:[26, 20, 28, 9, 11, 25]
Total times I have to pay if these scenes are scheduled (greedy): 28
Without Forcing 6 scenes per day:
Scenes of day 1:[5, 0, 2, 3, 7, 9]
Scenes of day 2:[13, 20, 27, 1, 4, 14]
Scenes of day 3:[19, 6, 10, 11, 15, 25]
Scenes of day 4:[16, 17, 28, 12, 8, 24]
Scenes of day 5:[21, 23, 26]
Scenes of day 6:[18, 29]
Scenes of day 7:[22]
Total times I have to pay if these scenes are scheduled (greedy): 44


In [45]:
# Source https://developers.google.com/optimization/scheduling/employee_scheduling?hl=es-419#scheduling_with_shift_requests
# I was trying with a solver to compare results, google implements something similar but for maximizing nurses shifts.
# Interesting enough the this solver returns the 27 as the best solution, same as the greedy algorithm above.
# With this solver I cant find a way to allow more than 5 days (num_scenees / max_scenes_per_day)

from ortools.sat.python import cp_model

# Sessions is scenes x actors
num_scenes, num_actors = sessions.shape
max_scenes_per_day = 6
max_days = int(num_scenes / max_scenes_per_day)


model = cp_model.CpModel()

# Variables for the solver
# scene_day[s][d] = 1 if scene s is assigned to day d
scene_day = []
for s in range(num_scenes):
    scene_day.append([model.NewBoolVar(f'scene_{s}_day_{d}') for d in range(max_days)])

# actor_day[a][d] = 1 if actor a is present on day d
actor_day = []
for a in range(num_actors):
    actor_day.append([model.NewBoolVar(f'actor_{a}_day_{d}') for d in range(max_days)])

# Constraints
# Each scene assigned to exactly one day
for s in range(num_scenes):
    model.Add(sum(scene_day[s][d] for d in range(max_days)) == 1)

# No more than X scenes per day
for d in range(max_days):
    model.Add(sum(scene_day[s][d] for s in range(num_scenes)) <= max_scenes_per_day)

# If an actor appears in any scene on a day, they must be present that day
for a in range(num_actors):
    for d in range(max_days):
        for s in range(num_scenes):
            if sessions[s, a] == 1:
                # If scene s is assigned to day d and actor a is in scene s, actor a must be present that day
                model.AddImplication(scene_day[s][d], actor_day[a][d])
        # If actor_day[a][d] is True, at least one scene assigned to day d must require actor a
        model.AddMaxEquality(actor_day[a][d], [scene_day[s][d] if sessions[s, a] == 1 else model.NewConstant(0) for s in range(num_scenes)])

# Objective: Minimize total actor-days
model.Minimize(sum(actor_day[a][d] for a in range(num_actors) for d in range(max_days)))

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)


if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    total_actor_days = int(solver.ObjectiveValue())
    # Print schedule
    for d in range(max_days):
        scenes_today = [s for s in range(num_scenes) if solver.Value(scene_day[s][d])]
        if scenes_today:
            print(f'Day {d+1}: Scenes {scenes_today}')
    print("Total times I have to pay if these scenes are scheduled (Solver):", total_actor_days)
    print("Solver result is:", solver.StatusName(status))

else:
    print('No solution found.')

Day 1: Scenes [2, 3, 7, 14, 20, 28]
Day 2: Scenes [5, 8, 12, 26, 27, 29]
Day 3: Scenes [9, 11, 15, 19, 24, 25]
Day 4: Scenes [0, 1, 4, 6, 10, 21]
Day 5: Scenes [13, 16, 17, 18, 22, 23]
Total times I have to pay if these scenes are scheduled (Solver): 27
Solver result is: OPTIMAL


(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>

¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?




Respuesta

Sin restricciones simplemente llevaria a todos los actores un mismo dia y haria todas las sesiones ese dia. Habria una sola posibilidad que seria todas las tomas juntas.

Con restricciones, de la manera que he implmentado el algoritmo hay muchas, al comienzo toma una random tengo 30 potenciales sesiones y luego se seleccionan las mejores opciones para los actores en esa escena y en las que se van agregando al dia. Ya que esto se hace sistematicametne de la misma manera habira 30 combinaciones para el 1er dia, leugo se repoite el proceso en los dias siguientes con las 24, 18, 12, 6 sessiones, con lo que la combinatoria seria el resuldato de 30 * 24 * 18 * 12 * 6

Las mejores son las que pagare 27 veces.

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

Yo defini sessions como un np.array, lo seleccione porque el archivo incial ya tiene un formato de nxm que facilmemte se puede representar de esta manera y es facil para iterar y correlacionar.

Según el modelo para el espacio de soluciones

(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?
Respuesta

La funcion objetivo es minimizar el número total de veces que se tiene que pagar a los actores por aparecer en las escenas asignadas a cada día.
Es un problema de minimización de costos

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [ ]:
# This is a brute force approach to find the best combination of scenes per day
# It generates all possible partitions of scenes into groups of up to max_per_day
# and calculates the cost for each partition, returning the one with the minimum cost

# This is used for generating all possible partitions of scenes into groups
import itertools 
import numpy as np

# Using a samll exaomple of scenes just to test the algorithm: 6 scenes, 3 actors
doblaje = np.array([
    [1, 0, 1],
    [1, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

num_scenes, num_actors = doblaje.shape
max_per_day = 3  # Testing with 3 scenes per day

# Generate all possible partitions of scenes into groups of up to max_per_day
def all_partitions(scenes, max_group_size):
    if not scenes:
        yield []
        return
    for i in range(1, min(len(scenes), max_group_size) + 1):
        for group in itertools.combinations(scenes, i):
            rest = [s for s in scenes if s not in group]
            for partition in all_partitions(rest, max_group_size):
                yield [list(group)] + partition

scenes = list(range(num_scenes))
min_cost = float('inf')
best_partition = None

# For each partition, calculate the cost (number of unique actors per day)
# and find the one with the minimum cost
for partition in all_partitions(scenes, max_per_day):
    cost = 0
    for day in partition:
        actors = set()
        for scene in day:
            actors |= set(np.where(doblaje[scene] == 1)[0])
        cost += len(actors)
    if cost < min_cost:
        min_cost = cost
        best_partition = partition

print("Best Scene Combination:", best_partition)
print("I have to pay:", min_cost)

Best Scene Combination: [[0, 1, 3], [2, 4, 5]]
I have to pay: 5


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta



Por lo que lei, si no se tuvieran restrincciones seria B(n) que corresponde a algo llamado Numero Bell, una serie que crece mas rapido que la exponencial: https://es.wikipedia.org/wiki/N%C3%BAmero_de_Bell 

Al tener las resticciones es calculable en muestras chicas, se tomaria una combinatoria de todas las posibles escenas (n) en grupos de Max_Scenes (k), entonces si llamamos a esto P(n, k) y:

n es el numero de escenas

a es el numero de actores

k es el numero maximo de escenas por dia

Seria O(n * a * P(n, k))

El problema es que P(n, k) crece como mas rapido que una exponencial y no seria calculable para 30 escenas.



(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

Entiendo que mi algoritmo iriginal es una mejora de fuerza bruta : scenes_organization_gh

El algoritmo propuesto consiste en seleccionar una seleccionar una escena de forma aleatoria y asignarle al mismo dia de rodado aquellas escena que compartan la mayor cantidad de actores.
Tambien acelero el tiempo de procesamiento eliminando las escenas ya seleccionadas de la lista para analisis, esto hace los ciclos cada vez nas cortos ya que hay menos escenas para analizar.

No estoy haciendo una combinatoria sino que estoy basandome en la escena aleatoria seleccionada, esto puede traer como consecuencia que la seed no sea la optima y el resultado sea mas alto que el ideal.

Mi algoritmo comparado con el de fuerza bruta no devuelve simpre la mejor opcion, sin embargo al ser computacionalmente mas rapido se puede correr muchas veces y seleccionar el mejor resultado (lo que he hecho). Al agregar este paso casi siempre se obtiene la mejor solucion y se resuleve en un tiempo computacional de segundos. Se podria tambien implementar una busqueda local pero ya que con probar 100 veces en 2 seg obtengo casi siempre el resultado optimo no vi motivo.

Si utlizaramos fuerza bruta para el caso planteado el proceso no terminaria nunca ya que la combinatoria de 30 escenas es tan grande que no habira forma de resolverla, incluso con la restriccion de 6 escenas por dia.

(*)Calcula la complejidad del algoritmo

Respuesta

Para cada dia, se selecciona una escena ( n ) semilla y se buscan ( k-1 ) escenas adicionales (siendo ( k ) el máximo de escenas por día, por ejemplo 6.). La cantidad de dias de rodado estara dada por n/k (En este caso 30/6 = 5 ciclos)

Para cada selección, se evalúan hasta escenas restantes ( n! ), y para cada una se calcula el numero de actores únicos ( a siendo a el número de actores ).

Por lo tanto, la complejidad es aproximadamente:

O(n/k × n! × a)






Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [33]:
def generate_random_scenes(num_scenes, num_actors):
    # Assign a random probability for each actor
    probs = np.random.uniform(0.1, 0.9, size=num_actors)

    # Initialize the doblaje matrix with zeros
    scenes = np.zeros((num_scenes, num_actors), dtype=int)

    # For each scene, randomly assign actors based on their individual probabilities
    for a in range(num_actors):
        scenes[:, a] = (np.random.rand(num_scenes) < probs[a]).astype(int)
    return scenes

# Test
scenes = generate_random_scenes(30, 10)

Aplica el algoritmo al juego de datos generado

Respuesta

In [42]:
print('Forcing always 6 scenes per day:')

scenes = generate_random_scenes(30, 10)
days = scenes_organization_gh(sessions, False)
total_actor_days = calc_cost(days_n,sessions)

day = 1
for day_scenes in days:
    print(f'Scenes of day {day}:{day_scenes}')
    day += 1

print("Total times I have to pay if these scenes are scheduled (greedy):", total_actor_days)

Forcing always 6 scenes per day:
Scenes of day 1:[19, 1, 12, 16, 18, 22]
Scenes of day 2:[21, 8, 27, 29, 0, 5]
Scenes of day 3:[23, 17, 13, 7, 9, 11]
Scenes of day 4:[26, 15, 2, 4, 3, 6]
Scenes of day 5:[20, 28, 10, 14, 25, 24]
Total times I have to pay if these scenes are scheduled (greedy): 42


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

[1] Wikipedia, “Número de Bell,” Wikipedia, La enciclopedia libre. [Online]. Available: https://es.wikipedia.org/wiki/N%C3%BAmero_de_Bell.

[2] Google Developers, “Scheduling with shift requests,” Google OR-Tools Documentation. [Online]. Available: https://developers.google.com/optimization/scheduling/employee_scheduling?hl=es-419#scheduling_with_shift_requests.


Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Podria testear mas a fondo el Solver propuesto o con otros, lo que tendria que explorar con mas profuncidad es la posibilidad de organizar las sesiones en mas de 5 dias (para 30 tomas) si es que hay alguna manera que sea mas barato.

Por ejemplo en algun caso donde 4 escenas tengan los mismos actores 5 y luego otras escenas 3 tengan un grupo diferente de 4 sera mejor convocar para un dia los 5 y otro los 4 y no grabar 6 escenas ningun dia.